<a href="https://colab.research.google.com/github/poet-developer/getPoemLineData_with-wikisource/blob/main/%EC%9C%84%ED%82%A4%EB%AC%B8%ED%97%8C_%ED%95%9C%EA%B0%9C%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 위키문헌(Wikisource) XML파일 처리

In [18]:
pip install -q wikiextractor gdown

In [19]:
# 필요 패키지 load
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
# import gutenbergpy.textget
from pprint import pprint
import gdown
from collections import Counter
from itertools import chain

### 위키문헌 (wikisource)
https://dumps.wikimedia.org/kowikisource/20240901/  
참고문헌 : 서재현, 김병준, 김민우 and 박소정. (2021). 멀리서 읽는 “우리”― Word2Vec, N-gram을 이용한 근대 소설 텍스트 분석. 대동문화연구, 115, 349-386.

##### 위키문헌 전처리

In [20]:
# 위키문헌 xml 원본 파일 다운로드
# https://dumps.wikimedia.org/kowikisource/20240901/kowikisource-20240901-pages-articles.xml.bz2
!wget https://dumps.wikimedia.org/kowikisource/20241101/kowikisource-20241101-pages-articles.xml.bz2

--2024-11-19 11:32:29--  https://dumps.wikimedia.org/kowikisource/20241101/kowikisource-20241101-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.71, 2620:0:861:3:208:80:154:71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151135574 (144M) [application/octet-stream]
Saving to: ‘kowikisource-20241101-pages-articles.xml.bz2.1’

kowikisource-202411 100%[===================>] 144.13M  4.31MB/s    in 34s     

2024-11-19 11:33:03 (4.30 MB/s) - ‘kowikisource-20241101-pages-articles.xml.bz2.1’ saved [151135574/151135574]



In [21]:
# Parsing (3분 가량 걸림)
!python -m wikiextractor.WikiExtractor kowikisource-20241101-pages-articles.xml.bz2

INFO: Preprocessing 'kowikisource-20241101-pages-articles.xml.bz2' to collect template definitions: this may take some time.
INFO: Loaded 1242 templates in 72.9s
INFO: Starting page extraction from kowikisource-20241101-pages-articles.xml.bz2.
INFO: Using 1 extract processes.
INFO: Finished 1-process extraction of 37057 articles in 156.5s (236.8 art/s)


In [22]:
# 파싱한 파일을 하나의 xml로 합치기
!cat text/*/* > kowikisource.xml

In [23]:
# 맨위에 추가
!echo "$(echo -n -e '<?xml version="1.0" encoding="UTF-8"?>\n<wikisource>\n'; cat kowikisource.xml)" > kowikisource.xml

In [24]:
!head -n 20 kowikisource.xml # 첫 20줄만 확인

<?xml version="1.0" encoding="UTF-8"?>
<wikisource>
<doc id="1" url="https://ko.wikisource.org/wiki?curid=1" title="애국가 (대한민국)">
애국가 (대한민국)

한자 혼용
한글

</doc>
<doc id="2" url="https://ko.wikisource.org/wiki?curid=2" title="대한민국헌법 (한자혼용)">
대한민국헌법 (한자혼용)



</doc>
<doc id="3" url="https://ko.wikisource.org/wiki?curid=3" title="조선민주주의인민공화국 사회주의헌법">
조선민주주의인민공화국 사회주의헌법

서문.
조선민주주의인민공화국은 위대한 김일성동지와 김정일동지의 사상과 령도를 구현한 주체의 사회주의조국이다.


In [25]:
# 맨 아랫줄에 추가
!echo '</wikisource>' >> kowikisource.xml

In [26]:
!tail kowikisource.xml # 마지막줄 확인



</doc>
<doc id="93454" url="https://ko.wikisource.org/wiki?curid=93454" title="에티카 (스피노자)">
에티카 (스피노자)



</doc>
</wikisource>


##### 위키문헌 작가의 시 행 추출

In [49]:
# xml 파일 dataframe로 읽기
df = pd.read_xml('./kowikisource.xml')

### 분리 행 추출 함수(getPoemLines)

In [28]:
def process_getPoemLines(data):
    # 불필요한 인자 제거
    filtered_list = [item for item in data if item not in [None, "", " ", [], {}, '<poem>', '</poem>', '진달래꽃 (시집)/']]

    # 제목 편집 (0번 인자 수정)
    pick_title = filtered_list[0].split('/')
    pick_title = pick_title[1] if len(pick_title) > 1 else filtered_list[0]
    filtered_list[0] = pick_title

    # 'title'은 첫 번째 인자, '본문'은 나머지 인자들
    title = filtered_list[0]
    body = filtered_list[1:]

    # DataFrame 생성: 본문 열이 먼저 오도록
    df = pd.DataFrame({'본문': body, '제목': [title] * len(body)})

    return df

### 시 전체 추출 함수(getPoemLines)

In [46]:
def process_getFullPoemLines(data):
    # 불필요한 인자 제거
    cleaned_text = data.replace('<poem>', '').replace('</poem>', '').strip()
    # 텍스트를 줄 단위로 나누기
    lines = cleaned_text.strip().split('\n')
    title = lines[0]  # 첫 줄은 제목
    body = '\n'.join(lines[1:])  # 나머지는 본문

    # DataFrame 생성
    df = pd.DataFrame({'제목': [title], '본문': [body]})

    return df

### 검색 데이터를 가지고 특정 시 행마다 추출(update_results_with_title)

In [29]:
def getOnlyOneresult_with_title(df, results_df, process_function, title_to_search, author, index):
    # 제목에 해당하는 텍스트 검색
    title_to_search.replace("_", " ")
    filtered_text = df[df['title'].str.contains(title_to_search, na=False)]
    if filtered_text.empty:
        raise ValueError(f"'{title_to_search}'에 해당하는 데이터가 없습니다.")

    # 'doc' 값 추출
    text_value = filtered_text.iloc[index]['doc']

    # 텍스트 처리
    processed_result = process_function(text_value.split('\n'))

    # 처리 결과를 기존 DataFrame에 추가
    updated_results = pd.concat([results_df, pd.DataFrame(processed_result)], ignore_index=True)

    # 새로운 열 추가 (고정된 값)
    updated_results['저자'] = author

    return updated_results

### 검색 데이터를 가지고 특정 시 전체 추출(update results for FullText)

In [35]:
def getOnlyOneFullTextresult_with_title(df, results_df, process_function, title_to_search, author, index):
    # 제목에 해당하는 텍스트 검색
    title_to_search.replace("_", " ")
    filtered_text = df[df['title'].str.contains(title_to_search, na=False)]
    if filtered_text.empty:
        raise ValueError(f"'{title_to_search}'에 해당하는 데이터가 없습니다.")

    # # 'doc' 값 추출
    full_text_value = filtered_text.iloc[index]['doc']

    # # 텍스트 처리
    processed_result = process_function(full_text_value)

    # # 처리 결과를 기존 DataFrame에 추가
    updated_results = pd.concat([results_df, pd.DataFrame(processed_result)], ignore_index=True)
    # 새로운 열 추가 (고정된 값)
    updated_results['저자'] = author

    return updated_results

In [32]:
#검색하고자 하는 제목과 자료 순서 입력.
title = '님의 침묵'
index = 0
author = '한용운'
#바르게 검색되었는지 확인.
check = df[df['title'].str.contains(title)]
check

,id,url,title,doc
30,1362,https://ko.wikisource.org/wiki?curid=1362,님의 침묵/가갸날,"\n님의 침묵/가갸날\n\n<poem>\n아아, 가갸날\n참되고 어질고 아름다와요\..."
59,1513,https://ko.wikisource.org/wiki?curid=1513,님의 침묵/님의 침묵,\n님의 침묵/님의 침묵\n\n원문.\n<poem>\n님은 갓슴니다 아々 사랑하는나...
68,1526,https://ko.wikisource.org/wiki?curid=1526,님의 침묵/나룻배와 행인,\n님의 침묵/나룻배와 행인\n\n<poem>\n나는 나룻배 \n당신은 行人\n당신...
281,2239,https://ko.wikisource.org/wiki?curid=2239,한용운의 님의 침묵,\n한용운의 님의 침묵\n\n\n\n
614,3646,https://ko.wikisource.org/wiki?curid=3646,님의 침묵/알 수 없어요,\n님의 침묵/알 수 없어요\n\n원문.\n<poem>\n바람도업는공중에 垂直의波紋...
...,...,...,...,...
9899,25036,https://ko.wikisource.org/wiki?curid=25036,님의 침묵/요술,\n님의 침묵/요술\n\n<poem>\n가을 洪水가 작은 시내의 쌓인 落葉을 휩쓸어...
9900,25037,https://ko.wikisource.org/wiki?curid=25037,님의 침묵/여름밤이 길어요,\n님의 침묵/여름밤이 길어요\n\n<poem>\n당신이 계실 때에는 겨울밤이 짧더...
9901,25038,https://ko.wikisource.org/wiki?curid=25038,님의 침묵/칠석,\n님의 침묵/칠석\n\n<poem>\n「차라리 님이 없이 스스로 님이 되고 살지언...
9902,25039,https://ko.wikisource.org/wiki?curid=25039,님의 침묵/오셔요,\n님의 침묵/오셔요\n\n<poem>\n오셔요 당신은 오실 때가 되었어요 어서 오...


In [42]:
#빈 프레임 만들기 (열은 본문과 title)
text = pd.DataFrame(columns=['본문', '제목'])

### 데이터 추출 (행마다)

In [33]:
# 작품 한개씩 행 추출 (데이터에 포함된 작품이름과, 작가 이름을 검색.)
text = getOnlyOneresult_with_title(df, text, process_getPoemLines, title, author, index) #검색어, 작가 이름 입력
text.iloc[-1] # 마지막줄 확인

,25
본문,"가갸날,오오 가갸날이여"
제목,가갸날
저자,한용운


### 데이터 추출 (전체)

In [47]:
# 작품 한개씩 행 추출 (데이터에 포함된 작품이름과, 작가 이름을 검색.)
text = getOnlyOneFullTextresult_with_title(df, text, process_getFullPoemLines, title, author, index) #검색어, 작가 이름 입력
text.iloc[0] # 마지막줄 확인

,0
본문,"\n\n아아, 가갸날\n참되고 어질고 아름다와요\n축일(祝日) 제일(祭日)\n메이 ..."
제목,님의 침묵/가갸날
저자,한용운


### 엑셀로 저장

In [48]:
# 엑셀로 저장하기
text.to_excel('위키문헌_한용운_행별.xlsx')